In [6]:
import numpy as np
from dataset import MatchingDataset
from tqdm import tqdm
import deepdish as dd
from colmap_utils.read_write_model import read_model

In [7]:
matching_dataset = MatchingDataset(["brandenburg_gate", "buckingham_palace", "sacre_coeur", "st_pauls_cathedral", "palace_of_westminster"], max_pairs_per_dataset=10000)

Extracting brandenburg_gate data...


Processing brandenburg_gate image pairs: 100%|██████████| 10000/10000 [17:16<00:00,  9.65it/s]


Extracting buckingham_palace data...


Processing buckingham_palace image pairs: 100%|██████████| 10000/10000 [17:43<00:00,  9.41it/s]


Extracting sacre_coeur data...


...

Extracting st_pauls_cathedral data...


Processing st_pauls_cathedral image pairs: 100%|██████████| 10000/10000 [17:49<00:00,  9.35it/s]


Extracting palace_of_westminster data...


Processing palace_of_westminster image pairs: 100%|██████████| 10000/10000 [15:49<00:00, 10.53it/s]

Samples with padding: 668/50000


In [8]:
no_matches = 0
padding_rows = 0
n = matching_dataset[0][0].shape[1]
keypoints = matching_dataset[0][0].shape[0]
freq = np.zeros(keypoints)

for input_data, label in tqdm(matching_dataset):
    matched = label[label != -1]
    no_matches += keypoints - len(matched)

    for match in matched:
        freq[int(match)] += 1

    for i in range(len(input_data) - 1, -1, -1):
        if np.any(input_data[i][: n // 2]) and np.any(input_data[i][n // 2:]):
            break
        if not np.any(input_data[i][: n // 2]):
            padding_rows += 1
        if not np.any(input_data[i][n // 2:]):
            padding_rows += 1

100%|██████████| 50000/50000 [00:15<00:00, 3150.57it/s]


In [9]:
print(f"Padded rows {padding_rows}/{keypoints * len(matching_dataset)}, {(padding_rows) / (keypoints * len(matching_dataset)) * 100}%")

Padded rows 146803/25600000, 0.57344921875%


In [10]:
print(f"Matched pairs of keypoints {keypoints * len(matching_dataset) - no_matches}/{keypoints * len(matching_dataset)}, {(1 - no_matches / (keypoints * len(matching_dataset))) * 100}%")

Matched pairs of keypoints 13580699/25600000, 53.049605468749995%


In [11]:
buildings = ["brandenburg_gate", "buckingham_palace", "sacre_coeur", "st_pauls_cathedral", "palace_of_westminster"]
total_images = 0
total_pairs = 0
valid_pairs = 0

for building in buildings:
    print(f"--------{building}--------")
    _, images, _ = read_model(path=building + "/dense/sparse" , ext='.bin')
    print(f'Images: {len(images)}')
    total_images += len(images)

    pairs = dd.io.load(building + '/dense/stereo/pairs-dilation-0.00-fixed2.h5')
    if len(pairs[list(pairs.keys())[0]]) == 3:
        idx1, idx2 = 0, 1
    else:
        idx1, idx2 = 2, 3
    total_pairs += len(pairs)

    filtered = []
    for p in pairs:
        if pairs[p][idx1] >= 0.4 and pairs[p][idx2] >= 0.4:
            filtered.append(p)
    
    print(f'Valid pairs: {len(filtered)}/{len(pairs)}')
    valid_pairs += len(filtered)
    pairs = filtered

--------brandenburg_gate--------
Images: 1363
Valid pairs: 224200/928203
--------buckingham_palace--------
Images: 1676
Valid pairs: 70225/1403650
--------sacre_coeur--------
Images: 1179
Valid pairs: 117313/694431
--------st_pauls_cathedral--------
Images: 615
Valid pairs: 84910/188805
--------palace_of_westminster--------
Images: 983
Valid pairs: 70746/482653
